In [2]:
from openai import OpenAI

In [28]:
import re

In [3]:
client = OpenAI()

In [4]:
chat_completions = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user", "content":"Hello how are you"}]
)
chat_completions.choices[0].message.content

"Hello! I am an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"

In [19]:
from typing import Any


class Agent:
    def __init__(self, message):
        self.message = [{"role":"system", "content":message}]
    
    def __call__(self, message):
        self.message.append({"role":"user", "content":message})
        completions = self.execute()
        self.message.append({"role":"assistant", "content":completions})
        
        return completions

    def execute(self):
        chat_completions = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = self.message
        ) 
        return chat_completions.choices[0] .message.content  
    



In [55]:
prompt = """
You have to give the weight of the Phone given as input in a loop of Thought, Action, Pause and Observation.
Each loop has exactly 1 action.
The thought is your explanation of the answer you are going to give.
The actions can be one of the following:
1. Calculate: 
eg: Calculate: 3 + 2 / 4
This returns the calculation of the given expression. Uses python to evaluate the expression. So floating point numbers are supported.

2. Find_Weight_Phone:
eg: Find_Weight_Phone: iPhone 14
This returns the weight of the phone given as input. Weight is in grams.

After the actions are chosen, Pause and return. The observation will be provided in the next input in the below format:
Observation: <Observation message>

Example of the complete flow:
Find the weight of the phone iPhone 15

First your message: I need to find the weight of the phone iPhone 15. Action: Find_Weight_Phone: iPhone 15
Then Pause
User will send the message: Observation: Weight of phone iPhone 15 = 171 grams

Give the output as: Weight of phone iPhone 15 is 171 grams

""".strip()

In [56]:
ag = Agent(prompt)

In [36]:
def Find_Weight_Phone(phone):
    if phone in "iPhone 14":
        weight =  172
    elif phone in "iPhone 15":
        weight = 171
    elif phone in "Samsung S23":
        weight = 168
    elif phone in "Samsung S24":
        weight = 167
        
    return f"Weight of phone is {weight}"

In [50]:
def Calculate(expr):
    return f"Calculation result is : {eval(expr)}"


In [52]:
Calculate("2 + 5 - 7 / 2")

'Calculation result is : 3.5'

In [21]:
completions = ag("Find the weight of iPhone 15")
completions

'Thought: I need to find the weight of the iPhone 15.\nAction: Find_Weight_Phone: iPhone 15\nPause'

In [22]:
Find_Weight_Phone("iPhone 15")

171

In [23]:
response = f"Observation: Weight of phone: iPhone 15 = 171"
completions = ag(response)
completions

'Weight of phone iPhone 15 is 171 grams'

In [24]:
available_fns = {"Find_Weight_Phone": Find_Weight_Phone, "Calculate": Calculate}

In [26]:
available_fns["Find_Weight_Phone"]("iPhone 15")

171

In [31]:
sample_completions = "Thought: I need to find the weight of the iPhone 15.\nAction: Find_Weight_Phone: iPhone 15\nPause"
action_re = re.compile("^Action: (\w+): (.*)$")
action_matches = action_re.match(sample_completions.split("\n")[1])
action, action_input = action_matches.groups()
print(f"action : {action} and action input : {action_input}")

action : Find_Weight_Phone and action input : iPhone 15


In [35]:
test_arr = []
if test_arr:
    print("Not empty")

In [44]:
def agent_perform(message, max_turns):
    ag = Agent(prompt)
    inp = message
    i = 0
    while i < max_turns:
        completions = ag(inp)
        print(completions)
        actions = [action_re.match(a) for a in completions.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action in available_fns.keys():
                response = f"Observation: {available_fns[action](action_input)}"
                inp = response
            else:
                return "Exception happened: unidentified actions"
        else:
            return
        i += 1

In [37]:
available_fns.keys()

dict_keys(['Find_Weight_Phone', 'Calculate'])

In [39]:
if "Find_Weight_Phone" in available_fns.keys():
    print("success")
else:
    print("failure")

success


In [47]:
input_to_test = "Find the weight of iPhone 15"
max_turns = 5
agent_perform(input_to_test, max_turns)

Thought: I need to find the weight of iPhone 15.
Action: Find_Weight_Phone: iPhone 15
Pause
Weight of iPhone 15 is 171 grams


In [48]:
input_to_test = "Find the weight of Samsung S24"
max_turns = 5
agent_perform(input_to_test, max_turns)

I need to find the weight of the phone Samsung S24. 

Action: Find_Weight_Phone: Samsung S24

Pause
Weight of phone Samsung S24 is 167 grams


In [58]:
input_to_test = "Find the average of weight of Samsung S24 and iPhone 15"
max_turns = 5
agent_perform(input_to_test, max_turns)

I need to find the average of the weight of Samsung S24 and iPhone 15. Action: Calculate: (160 + 171) / 2
Please Pause.
